<a href="https://colab.research.google.com/github/UpEon/project_j/blob/main/Ai_09_%EC%A0%84%EC%83%81%EC%96%B8_Section4_Project_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 구성
- sn - 데이터 번호
- file_name - 정제된 원본 문서명
- data_set - 기술과학/사회과학 구분
- domain - 데이터 대분야
- subdomain - 데이터 소분야
- source - 원문 출처
- ko - 한국어 문장
- mt - 기계번역 문장
- en - 영어 문장
- source_language - 원문 언어 코드
- target_language - 번역문 언어 코드
- license - 라이선스
- style - 문체

In [ ]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 필요 라이브러리

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

import pandas as pd
import urllib.request
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import shutil

import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
train_csv = '1113_social_train_set_1210529.csv'
val_csv = '1113_social_valid_set_151316.csv'

train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)

print(train_df.shape, val_df.shape)

(1210529, 13) (151316, 13)


In [ ]:
# 데이터 정리
train_df = train_df[:10000]
val_df = val_df[:4000]

train_df = train_df[["ko","en"]]
val_df = val_df[["ko","en"]]

In [ ]:
print(len(train_df))
train_df.head()

10000


,ko,en
0,제2항을 함께 두는 것이 적절한지에 대하여 논의 과정에서 의문이 제기되기도 하였다.,"In the course of the discussion, questions wer..."
1,문제는 거래가 끊어진 경우에는 실제 거래가 이뤄지지 않기 때문에 이러한 실거래가 자...,"If the transaction is cut off, the actual tran..."
2,이러한 공공 부문 이동기기 개인 영상정보 규율의 문제점을 개선하기 위해 보호 체계 ...,This discussed ways to improve the protection ...
3,그 내용은 부품 사업자가 쉽게 알 수 없고 관여할 수 없기 때문에 중간 부품 생산자...,Since the contents are not easily known and in...
4,매칭정보를 획득하게 되면 통상 ‘ARP spoofing’이라는 해킹기법을 사용한다.,"When matching information is obtained, a hacki..."


In [ ]:
print(len(val_df))
val_df.head()

4000


,ko,en
0,"국경 없는 인터넷의 특 성을 감안하여, 저작권법의 속지적인 특성에도 불구하고 미국 ...",Considering the type regarding the Internet wi...
1,학습자들이 제시한 건의사항을 해결하기 위해 다음과 같은 논의가 필요하다.,The following discussion is needed to resolve ...
2,2011년에 발생한 저축은행 부실사태 이후 관련 연구가 종종 이뤄지고 있다.,Some studies on the period after the savings b...
3,기업의 성장 측정 방식을 선택함에 있어 기업 규모에 따른 편향적인 결과가 도출되는 ...,In selecting a company's growth measurement me...
4,"1인 여행객 중 여성은 52.3%, 남성은 47.7%로 여성이 상대적으로 나 홀로 ...","Among solo travelers, 52.3% were female and 47..."


In [ ]:
questions = []
for sentence in train_df['ko']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [ ]:
answers = []
for sentence in train_df['en']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [ ]:
questions2 = []
for sentence in val_df['ko']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions2.append(sentence)

In [ ]:
answers2 = []
for sentence in val_df['en']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers2.append(sentence)

In [ ]:
print(questions[:5])
print(answers[:5])

['제2항을 함께 두는 것이 적절한지에 대하여 논의 과정에서 의문이 제기되기도 하였다 .', '문제는 거래가 끊어진 경우에는 실제 거래가 이뤄지지 않기 때문에 이러한 실거래가 자료조차 찾을 수 없다는 점이다 .', '이러한 공공 부문 이동기기 개인 영상정보 규율의 문제점을 개선하기 위해 보호 체계 및 법제 개선방안을 논의하였다 .', '그 내용은 부품 사업자가 쉽게 알 수 없고 관여할 수 없기 때문에 중간 부품 생산자의 판매 행위는 대단히 불안한 지위에 놓이게 된다 .', '매칭정보를 획득하게 되면 통상 ‘ARP spoofing’이라는 해킹기법을 사용한다 .']
['In the course of the discussion ,  questions were raised as to whether it was appropriate to put Paragraph 2 together .', 'If the transaction is cut off ,  the actual transaction price cannot even be found because the actual transaction will not take place .', 'This discussed ways to improve the protection system and legal system in order to improve such a problem of the regulation of personal image information on mobile devices in the public sector .', "Since the contents are not easily known and involved by the parts business operator ,  intermediate parts manufacturers' sales behavior is placed in a precarious position .", "When matching information is obtained ,  a ha

In [ ]:
# 서브워드텍스트인코더를 사용하여 질문과 답변을 모두 포함한 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
# 서브워드텍스트인코더를 사용하여 질문과 답변을 모두 포함한 단어 집합(Vocabulary) 생성
tokenizer2 = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions2 + answers2, target_vocab_size=2**13)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN2, END_TOKEN2 = [tokenizer2.vocab_size], [tokenizer2.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE2 = tokenizer2.vocab_size + 2

In [ ]:
print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [8094]
종료 토큰 번호 : [8095]
단어 집합의 크기 : 8096


In [ ]:
# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('Tokenized sample question: {}'.format(tokenizer.encode(questions[20])))

Tokenized sample question: [2372, 3, 151, 5708, 2358, 258, 125, 221, 28, 829, 1345, 2284, 17, 216, 24, 125, 221, 675, 7380, 112, 1]


In [ ]:
# 서브워드텍스트인코더 토크나이저의 .encode()와 decode() 테스트해보기

# 임의의 입력 문장을 sample_string에 저장
sample_string = questions[20]

# encode() : 텍스트 시퀀스 --> 정수 시퀀스
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode() : 정수 시퀀스 --> 텍스트 시퀀스
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [2372, 3, 151, 5708, 2358, 258, 125, 221, 28, 829, 1345, 2284, 17, 216, 24, 125, 221, 675, 7380, 112, 1]
기존 문장: 반면 ,  실적이 예상보다 부진한 기업의 주요 원인은 내수 부진이라고 응답하였다 .


In [ ]:
# 각 정수는 각 단어와 어떻게 mapping되는지 병렬로 출력
# 서브워드텍스트인코더는 의미있는 단위의 서브워드로 토크나이징한다. 띄어쓰기 단위 X 형태소 분석 단위 X
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

2372 ----> 반면
3 ---->  ,  
151 ----> 실
5708 ----> 적이 
2358 ----> 예상
258 ----> 보다 
125 ----> 부
221 ----> 진
28 ----> 한 
829 ----> 기업의 
1345 ----> 주요 
2284 ----> 원인
17 ----> 은 
216 ----> 내
24 ----> 수 
125 ----> 부
221 ----> 진
675 ----> 이라고 
7380 ----> 응답
112 ----> 하였다
1 ---->  .


기계 번역기 만들기

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
embedding_dim = 64
hidden_units = 64

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

NameError: name 'src_vocab_size' is not defined

In [ ]:
def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer='adam', loss=loss_function, metrics=[accuracy])

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 30

model.fit(dataset, batch_size=64, epochs=EPOCHS, validation_data= dataset_val)
print(results.history)

# 훈련 과정 시각화 (정확도)
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# 훈련 과정 시각화 (손실)
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

재시도중 시간부족

# 에러 발생한 LSTM

In [ ]:
train_df.en = train_df.en.apply(lambda x : '\t' + x + '\n')
val_df.en = val_df.en.apply(lambda x : '\t' + x + '\n')

train_df.sample(10)

,ko,en
4918,"재가입 시점에, 당신은 변경된 보장내용으로 가입하게 되고 사회적, 경제적 상황에 따...","\tAt the time of re-subscription, you will sub..."
2041,"가족 자본주의""는 글자 그대로 창출된 이윤을 재투자하여 이윤의 지속적인 축적을 보장...","\t""Family capitalism"" literally means the fami..."
1067,본 연구에서는 플립러닝에서 협력적 자기효능감이 학습 성과를 유의하게 예측하는지 살펴...,"\tIn this study, we would like to examine whet..."
3272,부동산세는 건물 또는 토지에 부과되며 지방자치체 조세수입의 가장 주요한 재원이다.,\tReal estate taxes are levied on buildings or...
9506,"경찰도 범죄수사에 관한 준칙을 제정하고 시행하고 있는데, 그것이 바로 범죄수사 규칙이다.",\tThe police are also enacting and implementin...
7802,"먼저, 전체 프로그램을 세부적으로 살펴보자면, 두산 인문 극장이라는 주제 아래 3개...","\tFirst of all, looking at the whole program i..."
7794,대부분 해금 등 국악 연주자의 시각으로 음악을 분석하거나 무용 이론 및 역사에 대한...,\tMost of them are focused on analyzing music ...
7530,"변경 계약의 취소는 명칭 그대로'계약'의 취소에 관한것이지, 변경 계약과 별도로 '...",\tAs the name of the change contract is cancel...
4646,은행거래에 따른 수수료 등의 비용을 줄일 수 있을 것이다.,\tIt will be possible to reduce the cost of ba...
4311,"오히려 오늘의 설치 미술에서 보듯이, 자기만의 독자적 컨셉으로 승부하는 개념 예술의...","\tRather, as seen in today's installation art,..."


In [ ]:
# 문자 집합 구축
ko_vocab = set()
for line in train_df.ko: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        ko_vocab.add(char)

en_vocab = set()
for line in train_df.en:
    for char in line:
        en_vocab.add(char)

In [ ]:
# 문자 집합 구축
ko_vocab2 = set()
for line in val_df.ko: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        ko_vocab2.add(char)

en_vocab2 = set()
for line in val_df.en:
    for char in line:
        en_vocab2.add(char)

In [ ]:
ko_vocab_size = len(ko_vocab)+1
en_vocab_size = len(en_vocab)+1
print('source 문장의 char 집합 :',ko_vocab_size)
print('target 문장의 char 집합 :',en_vocab_size)

source 문장의 char 집합 : 1329
target 문장의 char 집합 : 132


In [ ]:
ko_vocab = sorted(list(ko_vocab))
en_vocab = sorted(list(en_vocab))
print(ko_vocab[45:75])
print(en_vocab[45:75])

['O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']
['L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']


In [ ]:
ko_to_index = dict([(word, i+1) for i, word in enumerate(ko_vocab)])
en_to_index = dict([(word, i+1) for i, word in enumerate(en_vocab)])
print(ko_to_index)
print(en_to_index)

{' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, '<': 28, '=': 29, '>': 30, '?': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, ']': 59, '_': 60, '`': 61, 'a': 62, 'b': 63, 'c': 64, 'd': 65, 'e': 66, 'f': 67, 'g': 68, 'h': 69, 'i': 70, 'j': 71, 'k': 72, 'l': 73, 'm': 74, 'n': 75, 'o': 76, 'p': 77, 'q': 78, 'r': 79, 's': 80, 't': 81, 'u': 82, 'v': 83, 'w': 84, 'x': 85, 'y': 86, 'z': 87, '~': 88, '£': 89, '°': 90, '·': 91, 'ö': 92, 'ʻ': 93, 'ʼ': 94, 'α': 95, 'π': 96, '–': 97, '—': 98, '‘': 99, '’': 100, '“': 101, '”': 102, '․': 103, '‧': 104, '₂': 105, '⃝': 106, '℃': 107, 'Ⅰ': 108, 'Ⅱ': 109, 'Ⅲ': 110, 'Ⅳ': 11

In [ ]:
encoder_input = []

# 1개의 문장
for line in train_df.ko:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(ko_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[975, 19, 1253, 935, 1, 1251, 266, 1, 421, 365, 1, 184, 941, 1, 967, 969, 1249, 1011, 867, 1, 390, 1247, 872, 1, 341, 940, 1, 210, 973, 867, 743, 1, 940, 616, 941, 1, 975, 245, 417, 245, 408, 1, 1247, 879, 379, 15], [616, 975, 365, 1, 177, 489, 158, 1, 289, 855, 1013, 1, 199, 907, 867, 365, 1, 801, 975, 1, 177, 489, 158, 1, 941, 542, 1011, 1011, 1, 834, 245, 1, 455, 616, 867, 1, 941, 498, 1249, 1, 801, 177, 489, 158, 1, 954, 532, 981, 1039, 1, 1046, 935, 1, 778, 1, 863, 379, 365, 1, 971, 941, 379, 15], [941, 498, 1249, 1, 209, 209, 1, 683, 616, 1, 941, 414, 245, 245, 1, 169, 943, 1, 880, 731, 973, 673, 1, 235, 930, 940, 1, 616, 975, 971, 935, 1, 169, 746, 1247, 245, 1, 923, 1254, 1, 673, 1278, 1, 1059, 200, 1, 635, 1, 658, 975, 1, 169, 746, 647, 832, 935, 1, 341, 940, 1247, 879, 379, 15], [237, 1, 313, 906, 934, 1, 683, 1232, 1, 724, 862, 954, 158, 1, 787, 187, 1, 835, 1, 778, 1, 863, 201, 1, 212, 872, 1250, 1, 778, 1, 863, 245, 1, 455, 616, 867, 1, 999, 160, 1, 68

In [ ]:
decoder_input = []
for line in train_df.en:
  encoded_line = []
  for char in line:
    encoded_line.append(en_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 43, 78, 3, 84, 72, 69, 3, 67, 79, 85, 82, 83, 69, 3, 79, 70, 3, 84, 72, 69, 3, 68, 73, 83, 67, 85, 83, 83, 73, 79, 78, 15, 3, 81, 85, 69, 83, 84, 73, 79, 78, 83, 3, 87, 69, 82, 69, 3, 82, 65, 73, 83, 69, 68, 3, 65, 83, 3, 84, 79, 3, 87, 72, 69, 84, 72, 69, 82, 3, 73, 84, 3, 87, 65, 83, 3, 65, 80, 80, 82, 79, 80, 82, 73, 65, 84, 69, 3, 84, 79, 3, 80, 85, 84, 3, 50, 65, 82, 65, 71, 82, 65, 80, 72, 3, 21, 3, 84, 79, 71, 69, 84, 72, 69, 82, 17, 2], [1, 43, 70, 3, 84, 72, 69, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 73, 83, 3, 67, 85, 84, 3, 79, 70, 70, 15, 3, 84, 72, 69, 3, 65, 67, 84, 85, 65, 76, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 80, 82, 73, 67, 69, 3, 67, 65, 78, 78, 79, 84, 3, 69, 86, 69, 78, 3, 66, 69, 3, 70, 79, 85, 78, 68, 3, 66, 69, 67, 65, 85, 83, 69, 3, 84, 72, 69, 3, 65, 67, 84, 85, 65, 76, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 87, 73, 76, 76, 3, 78, 79, 84, 3, 84, 65, 75, 69, 3, 80, 76, 65, 67, 69, 17, 2], [1, 54, 72,

In [ ]:
decoder_target = []
for line in train_df.en:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(en_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[43, 78, 3, 84, 72, 69, 3, 67, 79, 85, 82, 83, 69, 3, 79, 70, 3, 84, 72, 69, 3, 68, 73, 83, 67, 85, 83, 83, 73, 79, 78, 15, 3, 81, 85, 69, 83, 84, 73, 79, 78, 83, 3, 87, 69, 82, 69, 3, 82, 65, 73, 83, 69, 68, 3, 65, 83, 3, 84, 79, 3, 87, 72, 69, 84, 72, 69, 82, 3, 73, 84, 3, 87, 65, 83, 3, 65, 80, 80, 82, 79, 80, 82, 73, 65, 84, 69, 3, 84, 79, 3, 80, 85, 84, 3, 50, 65, 82, 65, 71, 82, 65, 80, 72, 3, 21, 3, 84, 79, 71, 69, 84, 72, 69, 82, 17, 2], [43, 70, 3, 84, 72, 69, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 73, 83, 3, 67, 85, 84, 3, 79, 70, 70, 15, 3, 84, 72, 69, 3, 65, 67, 84, 85, 65, 76, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 80, 82, 73, 67, 69, 3, 67, 65, 78, 78, 79, 84, 3, 69, 86, 69, 78, 3, 66, 69, 3, 70, 79, 85, 78, 68, 3, 66, 69, 67, 65, 85, 83, 69, 3, 84, 72, 69, 3, 65, 67, 84, 85, 65, 76, 3, 84, 82, 65, 78, 83, 65, 67, 84, 73, 79, 78, 3, 87, 73, 76, 76, 3, 78, 79, 84, 3, 84, 65, 75, 69, 3, 80, 76, 65, 67, 69, 17, 2], [54, 72, 73, 

In [ ]:
max_ko_len = max([len(line) for line in train_df.ko])
max_en_len = max([len(line) for line in train_df.en])
print('source 문장의 최대 길이 :',max_ko_len)
print('target 문장의 최대 길이 :',max_en_len)

source 문장의 최대 길이 : 192
target 문장의 최대 길이 : 463


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_ko_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_en_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_en_len, padding='post')

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

### seq2seq 기계 번역기 훈련시키기

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, ko_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, en_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(en_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=30, validation_split=0.2)

Epoch 1/40
250/250 [==============================] - 1121s 4s/step - loss: 1.1717 - val_loss: 0.8423
Epoch 2/40
250/250 [==============================] - 1112s 4s/step - loss: 0.7346 - val_loss: 0.6631
Epoch 3/40
250/250 [==============================] - 1169s 5s/step - loss: 0.6234 - val_loss: 0.5827
Epoch 4/40
250/250 [==============================] - 1164s 5s/step - loss: 0.5519 - val_loss: 0.5219
Epoch 5/40
250/250 [==============================] - 1177s 5s/step - loss: 0.4988 - val_loss: 0.4776
Epoch 6/40
250/250 [==============================] - 1158s 5s/step - loss: 0.4602 - val_loss: 0.4476
Epoch 7/40
250/250 [==============================] - 1144s 5s/step - loss: 0.4322 - val_loss: 0.4226
Epoch 8/40
250/250 [==============================] - 1137s 5s/step - loss: 0.4112 - val_loss: 0.4059
Epoch 9/40
250/250 [==============================] - 1140s 5s/step - loss: 0.3950 - val_loss: 0.3939
Epoch 10/40
250/250 [==============================] - 1135s 5s/step - loss: 0.382

### seq2seq 기계 번역기 동작시키기

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', train_df.ko[seq_index])
  print('정답 문장:', train_df.en[seq_index][2:len(train_df.en[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: 그 내용은 부품 사업자가 쉽게 알 수 없고 관여할 수 없기 때문에 중간 부품 생산자의 판매 행위는 대단히 불안한 지위에 놓이게 된다.
정답 문장: Since the contents are not easily known and involved by the parts business operator, intermediate parts manufacturers' sales behavior is placed in a precarious position.

번역 문장: The problem of a study is to examine the relationship between the two processes and the concept of a company and the concept of a construction of the concept of a construction of the concept of a construction of the concept of a construction of the concept of a constitution.
-----------------------------------
입력 문장: 먼저, 예제를 통해 정확한 목표 구역의 중요성을 살펴본다.
정답 문장: First, the importance of accurate lot targeting is reviewed through an example.

번역 문장: The problem of a study is to examine the relationship between the two processes and the concept of a company and the concept of a construction of the concept of a construction of the concept of a construction of the concept of a construction of the c

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 1478)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, None, 157)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        1776640     ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                 